# 1 Open and Import Dependencies

In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
from calculate_angle import calculate_angle
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 2. Determining Joints and check angle with timer

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [17]:
cap = cv2.VideoCapture('Sample.mp4')
s= "Bend your knee"

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

         # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            print(angle)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )

            if angle <70:
                 s= "Hold position for 8 Sec"
            

      
        except:
            pass
        
        #Render Countdown
        cv2.rectangle(image, (0,0), (250,100), (245,117,16), -1)
        cv2.putText(image, str(timer),
                    (40,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
       
                    

        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )                              
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(8) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

167.9889202055329
171.92984789675035
166.24777436102627
161.37202764564256
160.97895686508934
159.34205807482195
157.64258848085902
156.72002956361595
159.10066141388296
159.81863949432014
160.28645094735137
159.2116574974123
159.12752553060145
160.22916062584972
161.34897419125124
161.44734890014846
163.04438538892515
175.78876444774932
176.19836524332422
177.45127469266535
178.82254102925262
171.13497710578872
169.58993483685344
169.42700790655655
170.12707446042532
170.79355687557495
172.01539936463556
178.43510604208163
176.04248531884303
165.25241780115263
161.04971585078968
161.79811984069102
167.80226986986293
169.6841061450588
173.42027643269222
179.29224730262433
177.65129069583858
177.24986224082068
179.47488946840932
164.7413543250891
159.34192232553082
148.22501285967996
147.0430244152609
150.0915207439409
154.3610470026834
153.91046083770627
140.25126235624396
150.29065492511563
153.93679350880444
154.11442841951447
154.03623413381183
154.3179164644325
154.7332346844374
15